In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [2]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
import tensorflow as tf

In [6]:
!pip install -q tensorflow-hub
import tensorflow_hub as hub

In [7]:
target = train_data.pop('target')
target

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [8]:
tweets = train_data.iloc[:,3]
tweets

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((tweets.values, target.values))
dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [10]:
test_tweets = test_data.pop('text')
test_tweets

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259    Storm in RI worse than last hurricane. My city...
3260    Green Line derailment in Chicago http://t.co/U...
3261    MEG issues Hazardous Weather Outlook (HWO) htt...
3262    #CityofCalgary has activated its Municipal Eme...
Name: text, Length: 3263, dtype: object

In [11]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_tweets.values))
test_dataset

<TensorSliceDataset shapes: (), types: tf.string>

In [12]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data.iloc[:3,3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.00723878,  0.15689617,  1.2953067 , -0.04077986, -0.16036902,
        -1.2004429 , -0.40451345, -0.5239726 , -0.23558664, -0.35022184,
        -0.28988177, -0.47197863, -0.07621863,  0.17048942, -0.8323773 ,
         0.02260084,  0.16125458, -0.64086777, -0.569074  ,  0.04000134],
       [-0.4983734 ,  1.2828879 , -0.18836124, -0.94734734,  0.8805808 ,
        -0.17885518, -0.01543166,  0.3832704 , -1.508848  ,  1.7532811 ,
        -0.75493276,  0.60124093, -0.14378418, -0.891944  ,  0.43213663,
         1.3467408 , -1.4859253 ,  1.5762157 , -0.37878054,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 ,  1.7130805 ,  1.3203121 ,
        -3.201303  ,  0.8445131 ,  0.9621795 , -1.5353138 ,  2.2069528 ,
        -0.2790253 ,  0.01095965, -2.2282734 ,  0.2043116 , -2.17501   ,
         1.3441613 ,  0.11884003, -1.2388006 , -0.25501075,  0.69527215]],
      dtype=float32)>

In [13]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(dataset.shuffle(7612).batch(512), 
         epochs=20,verbose=1)

Train for 15 steps
Epoch 1/20
15/15 [==============================] - 1s 63ms/step - loss: 0.7095 - accuracy: 0.5287
Epoch 2/20
15/15 [==============================] - 0s 15ms/step - loss: 0.6451 - accuracy: 0.6482
Epoch 3/20
15/15 [==============================] - 0s 14ms/step - loss: 0.6014 - accuracy: 0.7008
Epoch 4/20
15/15 [==============================] - 0s 15ms/step - loss: 0.5686 - accuracy: 0.7414
Epoch 5/20
15/15 [==============================] - 0s 14ms/step - loss: 0.5440 - accuracy: 0.7549
Epoch 6/20
15/15 [==============================] - 0s 14ms/step - loss: 0.5241 - accuracy: 0.7619
Epoch 7/20
15/15 [==============================] - 0s 14ms/step - loss: 0.5082 - accuracy: 0.7686
Epoch 8/20
15/15 [==============================] - 0s 14ms/step - loss: 0.4946 - accuracy: 0.7738
Epoch 9/20
15/15 [==============================] - 0s 14ms/step - loss: 0.4820 - accuracy: 0.7831
Epoch 10/20
15/15 [==============================] - 0s 14ms/step - loss: 0.4699 - accurac

In [16]:
results = model.predict(test_tweets)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.963
accuracy: 0.809


In [17]:
results

array([[0.96320754],
       [0.80910754],
       [0.96635467],
       ...,
       [0.5971115 ],
       [0.8221302 ],
       [0.6732674 ]], dtype=float32)

In [18]:
output = []
for value in results:
    if value > 0.5:
        output.append(1)
    else:
        output.append(0)

In [19]:
output

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,


In [20]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [21]:
submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [22]:
submission['target'] = output
submission.to_csv("output.csv", index=False)
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
